In [1]:
from pynq import Overlay, DefaultIP
from IPython.display import HTML
from pynq import Xlnk
import numpy as np
from time import sleep


In [2]:
class QLearningDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
        
    @property
    def alpha(self):
        return self.read(28)
    @alpha.setter
    def alpha(self, alpha):
        self.write(8, alpha)
        
    @property
    def gamma(self):
        return self.read(32)
    @gamma.setter
    def gamma(self, gamma):
        self.write(12, gamma)
        
    @property
    def episodes(self):
        return self.read(20)
    
    def enable(self):
        self.write(4, 1)
    
    def disable(self):
        self.write(4, 0)
        
    bindto = ['xilinx.com:module_ref:qlearning_axi:1.0']
    
def policy(data):
    ctr = 0
    for da in data:
        
        u = int(da) >> 48
        r = (int(da) >> 32) & 0xFFFF
        d = (int(da) >> 16) & 0xFFFF
        l = int(da) & 0xFFFF
        print(ctr, u, r, d, l)
        ctr += 1
        
def policy(data):
    ctr = 0
    for da in data:
        u = int(da) >> 48
        r = (int(da) >> 32) & 0xFFFF
        d = (int(da) >> 16) & 0xFFFF
        l = int(da) & 0xFFFF
        print(ctr, u, r, d, l)
        ctr += 1

In [3]:
overlay = Overlay('design_1_wrapper.bit')
qlearn = overlay.qlearning_axi_0.S_AXI
dma = overlay.axi_dma_0

In [4]:
qlearn.gamma = 1
qlearn.alpha = 1

print(qlearn.gamma)
print(qlearn.alpha)
print(qlearn.episodes)

1
1
0


In [14]:
qlearn.enable()
sleep(1)
qlearn.disable()
qlearn.episodes

300687885

In [6]:
print(dma.sendchannel.running)
print(dma.recvchannel.running)

xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(16,), dtype=np.uint64)

True
True


In [17]:
dma.recvchannel.transfer(in_buffer)
dma.recvchannel.wait()
print(in_buffer)
policy(in_buffer)

[  281487861809153   281505041874947   563010083880966  1688978709413902
  3940915962314766   844485060657154  1688983006216198  3940915962314766
  1688909990723586  1688978711248903  3940915965984783  8444794763083806
  3940709805457414  3940778527031310  8444515597615134 72058689271300351]
0 1 3 3 1
1 1 7 6 3
2 2 14 14 6
3 6 30 2 14
4 14 62 6 14
5 3 14 15 2
6 6 31 30 6
7 14 62 6 14
8 6 14 14 2
9 6 30 30 7
10 14 62 62 15
11 30 127 14 30
12 14 14 30 6
13 14 30 62 14
14 30 62 127 30
15 256 255 255 255


In [9]:
dmaaxi = overlay.axi_dma_0.mmio
print(dmaaxi.read(0x34))
print(dmaaxi.read(0x30))

4098
65539
